# Scraping images from mapbox satelitar view  
Mapbox Free tier allows 50,000 requests per month : consider logging how many images have been scraped already to not exceed this limit.  


Pipeline : 
- Define area of interest (bounding box) from coordinates (lat, lon)
- Get the satelitar view of the bounding box from mapbox API
- Crop the images to remove watermark
- Save the images to disk
- Get the bounding boxes of buildings from OSM for the area of interest
- Save the bounding boxes to disk
- Clean overlapping bounding boxes (remove smaller ones or merge them)
- Save the cleaned bounding boxes to disk

The areas of interest have been picked manually :
- Top 20 most populous cities in France
- 100 rural areas coordinates picked randomly across France
- Maybe some random forest to get images without buildings (negative samples)

For the cities : 
- We define a bounding box around the city center with a fixed size (e.g., 10km x 10km)
- We then take random points from this bounding box to scrape images

Then, we can use the pipeline with these points to scrape images and get bounding boxes of buildings.

importing packages

In [16]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

CONFIG

In [17]:
# Load environment variables from .env file
load_dotenv()

# MAPBOX
MAPBOX_ACCESS_TOKEN = os.getenv("MAPBOX_ACCESS_TOKEN")

# Image parameters
WIDTH = 512
HEIGHT = 512 + 30  # extra space to then delete bottom part with watermark
ZOOM = 18  # 18-19 to see buildings, swimming pools clearly

PATH = Path().cwd().parent
BASE_DIR = PATH / "data"
RAW_DIR = BASE_DIR / "raw_images"
CROPPED_DIR = BASE_DIR / "cropped_images"

RAW_POLYGON_DIR = BASE_DIR / "raw_polygons"
CLEANED_POLYGON_DIR = BASE_DIR / "cleaned_polygons"

CONFIG_DIR = BASE_DIR / ".coordinates_and_mapping"

sys.path.append(str(PATH))

First, we get all the coordinates of the areas of interest from the two csv files.

In [18]:
# to import csv files
import pandas as pd

Rural areas coordinates file : "rural_coords.csv"

In [19]:
RURAL_COORDS_FILE_NAME: Path = CONFIG_DIR / "rural_coords.csv"
rural_coords: pd.DataFrame = pd.read_csv(
    RURAL_COORDS_FILE_NAME,
    sep = ",",
    header=None
)
rural_coords.columns = ["latitude", "longitude"]

In [20]:
# checking the rural coordinates file
print(f"Number of rural areas: {len(rural_coords)}")
rural_coords.head()

Number of rural areas: 100


Number of rural areas: 100


,latitude,longitude
0,45.534732,5.164968
1,44.947250,0.813114
2,45.534913,5.180068
3,44.946466,0.804980
4,45.534471,5.185063


Cities coordinates file : "city_coords.csv"

In [21]:
CITY_COORDS_FILE_NAME: Path = CONFIG_DIR / "city_coords.csv"
city_coords: pd.DataFrame = pd.read_csv(
    CITY_COORDS_FILE_NAME,
    sep = ","
)

In [22]:
# checking the city coordinates file
print(f"Number of cities: {len(city_coords)}")
city_coords.head()

Number of cities: 20


Number of cities: 20


,city,lat,lng
0,Paris,48.8567,2.3522
1,Bordeaux,44.8400,-0.5800
2,Marseille,43.2964,5.3700
3,Lyon,45.7600,4.8400
4,Toulouse,43.6045,1.4440


For the cities, we now create a bounding box around the city center with a fixed size (e.g., 10km x 10km) and then take random points from this bounding box to scrape images.

In [23]:
from miscellaneous import create_bbox_from_city_coordinates

city_coords_bbox: dict[str, dict[str, float]] = {}
# {city :  {"west": .., "south": .., "east": .., "north": ..}

for _, row in city_coords.iterrows():
    city_name: str = row["city"]
    latitude: float = row["lat"]
    longitude: float = row["lng"]
    bbox: dict[str, float] = create_bbox_from_city_coordinates(
        latitude,
        longitude,
        width=10,
        height=10
    )
    # width and height are in km
    city_coords_bbox[city_name] = bbox


In [24]:
city_coords_df = pd.DataFrame.from_dict(city_coords_bbox, orient="index")
city_coords_df.head()

,west,south,east,north
Paris,2.284676,48.811347,2.419842,48.902013
Bordeaux,-0.640288,44.792973,-0.519611,44.886994
Marseille,5.306649,43.252774,5.433447,43.339990
Lyon,4.774246,45.716018,4.905859,45.803944
Toulouse,1.383371,43.558618,1.504726,43.650350


Now that we have the bounding boxes for each city, we can get random points from these bounding boxes to scrape images.

In [25]:
from miscellaneous import get_random_points_in_bbox

city_random_points: list[dict[str, float]] = []
for city, bbox in city_coords_bbox.items():
    random_points: list[dict[str, float]] = get_random_points_in_bbox(
        bbox,
        number_of_points=10
    )
    # 10 coordinates/images for each city -> 20 * 10 = 200 images
    city_random_points.extend(random_points)
city_random_points_df = pd.DataFrame(city_random_points)
city_random_points_df.columns = ["latitude", "longitude"]

In [26]:
city_random_points_df.head()

,latitude,longitude
0,48.857595,2.363687
1,48.837874,2.392508
2,48.852587,2.294215
3,48.871388,2.380744
4,48.894442,2.375735


Now that we have all the coordinates (rural + cities), we combine them into a single list of coordinates to use in the pipeline.

In [27]:
# we combine both rural and city coordinates into a single dataframe
all_coords_df = pd.DataFrame(columns=["latitude", "longitude"])
all_coords_df = pd.concat(
    [
        rural_coords,
        city_random_points_df
    ],
    ignore_index=True
)

In [28]:
all_coords_df.head()

,latitude,longitude
0,45.534732,5.164968
1,44.947250,0.813114
2,45.534913,5.180068
3,44.946466,0.804980
4,45.534471,5.185063


We can now follow the pipeline to scrape images and get bounding boxes of buildings.

In [29]:
# testing on 5 random coordinates from the combined dataframe
coords_to_retrieve = all_coords_df
coords_to_retrieve = pd.read_csv(CONFIG_DIR / "coords_to_retrieve.csv")


Full pipeline

In [30]:
from miscellaneous import ask_mapbox_for_image, clean_overlapping_bboxes, create_bbox_from_coordinates
from PIL import Image
import osmnx as ox
import json

# first we import the OpenStreetMap mapping dict we created to have the mapping between OSM tags and our labels
with open(CONFIG_DIR / "osm_to_category_mapping.json", "r") as f:
    mapping_dict = json.load(f)

n = len(coords_to_retrieve)
print(f"Number of coordinates to retrieve: {n}")
for i, row in coords_to_retrieve.iterrows():
    # ====================================================================
    # 1. now we get bbox from coordinates to get the corresponding image from mapbox static api
    bbox: dict[str, float] = create_bbox_from_coordinates(
        row['latitude'],
        row['longitude']
    )
    # other parameters include :
    # - img_height: int = 512 + 30,
    # - img_width: int = 512,
    # - pixel_size: float = 0.4
    # returns : {"west": .., "south": .., "east": .., "north": ..}

    # ====================================================================
    # 2. we now use this bbox to retrieve the image from mapbox static api
    image = ask_mapbox_for_image(
        image_width_height={"width": WIDTH, "height": HEIGHT},
        mapbox_token=MAPBOX_ACCESS_TOKEN,
        bounding_box=bbox,
        request_timeout=10,
        output_file=RAW_DIR / f"image_{i+1}.png",  # if output_file is provided, the image will be saved at this location
    )

    # ====================================================================
    # 3. we now crop the image to remove the watermark at the bottom
    box = (0, 0, WIDTH, HEIGHT - 30)  # (left, upper, right, lower)
    cropped_image = image.crop(box)
    cropped_image.save(CROPPED_DIR / f"image_{i+1}.png")

    # ====================================================================
    # 4. now we can also get the bbox for each coordinate to get the corresponding polygon later on
    bbox_polyg: dict[str, float] = create_bbox_from_coordinates(
        row['latitude'],
        row['longitude'],
        img_height=512,
        img_width=512,
        pixel_size=0.4
    )

    # ===================================================================
    # 5. now we can also get the corresponding OSM tags for each coordinate to get the corresponding polygon later on
    # we will use the mapping dict to get the corresponding label for each coordinate
    bbox_for_ox = (bbox_polyg["west"], bbox_polyg["south"], bbox_polyg["east"], bbox_polyg["north"])
    features_from_bbox = ox.features_from_bbox(bbox=bbox_for_ox, tags={"building": True})

    # we add our category labels to the features
    features_from_bbox["category_of_building"] = features_from_bbox["building"].map(mapping_dict)
    # saving the features as a geojson file
    features_from_bbox.to_file(RAW_POLYGON_DIR / f"features_{i+1}.geojson", driver="GeoJSON")

    # ===================================================================
    # 6. we convert polygons to bounding boxes while keeping all metadata
    gdf_bbox = features_from_bbox.copy()
    gdf_bbox["geometry"] = gdf_bbox["geometry"].envelope

    # ===================================================================
    # 7. we now clean the overlapping bounding boxes
    gdf_cleaned = clean_overlapping_bboxes(gdf_bbox, threshold=0.3)
    # saving the cleaned features as a geojson file
    gdf_cleaned.to_file(CLEANED_POLYGON_DIR / f"cleaned_features_{i+1}.geojson", driver="GeoJSON")

    print(f"DONE for feature {i+1}, {n - (i+1)} remaining")

print("Pipeline completed successfully!")

Number of coordinates to retrieve: 5
DONE for feature 1, 4 remaining
DONE for feature 2, 3 remaining
DONE for feature 3, 2 remaining
DONE for feature 4, 1 remaining
DONE for feature 5, 0 remaining
Pipeline completed successfully!
